In [1]:
import dis
import inspect
from textwrap import dedent

from codetransformer.utils.pretty import a as show_ast, d as show_disassembly
  
import pytenn2016
from pytenn2016 import diagrams
from pytenn2016.tokens import show_tokens

# Clean bytecode caches so that importing rot13 fails the first time.
# This is using IPython's magic interpolation syntax.
!rm {pytenn2016.__file__.replace('__init__.py', '__pycache__')}/*

# Redraw diagrams
diagrams.draw_all()

Drawing /home/ssanderson/projects/pytenn2016/notebooks/images/compiler.svg
Drawing /home/ssanderson/projects/pytenn2016/notebooks/images/decoder.svg
Drawing /home/ssanderson/projects/pytenn2016/notebooks/images/importhook.svg
Drawing /home/ssanderson/projects/pytenn2016/notebooks/images/bytecode.svg


 <center>
  <h1>Unspeakably Evil Hacks in Service of Marginally-Improved Syntax</h1><br><br>
  <h2>Compile-Time Metaprogramming in Python</h2><br><br>
  <h3><a href="https://github.com/ssanderson/pytenn2016">https://github.com/ssanderson/pytenn2016</a>
  </h3><br>
  
</center>

# About Me:

<img src="images/me.jpg" alt="Drawing" style="width: 300px;"/>

- API Design Lead at [Quantopian](www.quantopian.com)
- Background in Mathematics and Philosophy
- **Twitter:** [@ssanderson11235](https://twitter.com/ssanderson11235)
- **GitHub:** [ssanderson](github.com/ssanderson)

# Outline

- "Standard" Metaprogramming Techniques
- Python Program Representations
- Custom File Encodings
- Import Hooks
- Bytecode Rewriting
- Philosophy

- Memes!

# Metaprogramming

> Metaprogramming is the writing of computer programs with the ability to treat programs as their data. It means that a program could be designed to read, generate, analyse or transform other programs, and even modify itself while running. - Wikipedia

<img alt="Xzibit loves metaprogramming!" src="images/yo-dawg.jpg" style="width: 1000px"/>

# Decorators

In [2]:
def noisey_add(a, b):
    print("add called with args: {args}".format(args=(a, b)))

    return a + b

...

def noisey_save(s):
    print("save called with args: {args}".format(args=(s,)))
    
    # /dev/null is web scale
    with open('/dev/null', 'w') as f:
        f.write(s)

noisey_add(1, 2)
noisey_save('Important Data')

add called with args: (1, 2)
save called with args: ('Important Data',)


In [3]:
from functools import wraps

def noisey(f):
    "A decorator that prints arguments to a function before calling it."
    name = f.__name__
    
    @wraps(f)
    def print_then_call_f(*args):
        print("{f} called with args: {args}".format(f=name, args=args))
        return f(*args)
    
    return print_then_call_f

In [4]:
@noisey
def add(a, b):
    return a + b

@noisey
def save(s):
    # Still web scale
    with open('/dev/null', 'w') as f:
        f.write(s)

add(1, 2)
save("Important Data")

add called with args: (1, 2)
save called with args: ('Important Data',)


# Metaclasses

In [5]:
import math

class Vector:
    "A 2-Dimensional vector."

    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def magnitude(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    def doubled(self):
        return type(self)(self.x * 2, self.y * 2)
    
v0 = Vector(1, 2)
print("Magnitude: %f" % v0.magnitude())
print("Doubled Magnitude: %f" % v0.doubled().magnitude())

Magnitude: 2.236068
Doubled Magnitude: 4.472136


In [6]:
class PropertyVector:
    "A 2-Dimensional vector, now with 100% fewer parentheses!"
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @property
    def magnitude(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    @property
    def doubled(self):
        return type(self)(self.x * 2, self.y * 2)
    
v1 = PropertyVector(1, 2)
print("Magnitude: %f" % v1.magnitude)
print("Doubled Magnitude: %f" % v1.doubled.magnitude)

Magnitude: 2.236068
Doubled Magnitude: 4.472136


In [7]:
import inspect
from pprint import pformat

# Our metaclass will automatically convert anything with this signature
# into a property.
property_signature = inspect.FullArgSpec(
    args=['self'], varargs=None, varkw=None, defaults=None, 
    kwonlyargs=[], kwonlydefaults=None, annotations={},
)

class AutoPropertyMeta(type):
    """Metaclass that wraps no-argument methods in properties."""
    
    def __new__(mcls, name, bases, clsdict):
        for name, class_attr in clsdict.items():
            try:
                signature = inspect.getfullargspec(class_attr)
            except TypeError:
                continue

            if signature == property_signature:
                print("Wrapping %s in a property." % name)
                clsdict[name] = property(class_attr)

        return super().__new__(mcls, name, bases, clsdict)

In [8]:
class AutoPropertyVector(metaclass=AutoPropertyMeta):
    "A 2-Dimensional vector, now with 100% less @property calls!"
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def magnitude(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    def doubled(self):
        return type(self)(self.x * 2, self.y * 2)

v2 = AutoPropertyVector(1, 2)
print("")
print("Magnitude: %f" % v2.magnitude)
print("Doubled Magnitude: %f" % v2.doubled.magnitude)

Wrapping doubled in a property.
Wrapping magnitude in a property.

Magnitude: 2.236068
Doubled Magnitude: 4.472136


# `exec`

"The Swiss Army Knife of Metaprogramming"

![Metaprogramming is the best](images/unicorn.jpg)

# That's all great but...

- Abstractions often incur runtime overhead.

- Certain operations can't be overloaded (e.g. ``is`` and ``not``, or catching exceptions).

- No support for syntactic extensions.
  - Can't add new syntax.
  - Often can't repurpose existing syntax.

- Doesn't quite feel *evil* enough...

<img alt="We have to go deeper!" src="images/leo.jpg" style="width: 1000px"/>

# Let's look at a complicated function...

In [9]:
def addtwo(a):
    return a + 2

addtwo(1)

3

# What Happened When I Hit Enter?

<img src="images/compiler.svg" width="3000"/>

In [10]:
raw_source = b"""\
def addtwo(a):
    return a + 2
    
addtwo(1)
"""

raw_source
# list(raw_source)

b'def addtwo(a):\n    return a + 2\n    \naddtwo(1)\n'

In [11]:
# Bytes to Text
import codecs

decoded_source = codecs.getdecoder('utf-8')(raw_source)[0]

print(decoded_source)

def addtwo(a):
    return a + 2
    
addtwo(1)



In [12]:
# Text to AST
import ast
syntax_tree = ast.parse(decoded_source)

body = syntax_tree.body
show_ast(body[1])

Expr(
  value=Call(
    func=Name(id='addtwo', ctx=Load()),
    args=[
      Num(1),
    ],
    keywords=[],
    starargs=None,
    kwargs=None,
  ),
)


In [13]:
# AST -> Bytecode
code = compile(syntax_tree, 'pytenn2016', 'exec')

code
# show_disassembly(code)

<code object <module> at 0x7fef4c11d300, file "pytenn2016", line 1>


<img src="images/scheming.jpg" width="1000"/>

# What can I muck with?
<img src="images/compiler.svg" width="3000"/>

# Custom Source Encodings

<img src="images/decoder.svg" width="2500"/>

In [14]:
!cat ../pytenn2016/rot13.py

# encoding: pytenn2016-rot13

qrs uryyb():
    cevag("Uryyb Ebgngrq Jbeyq!")


## Python doesn't know about our encoding...

In [15]:
from pytenn2016.rot13 import hello 

hello()

SyntaxError: unknown encoding for '/home/ssanderson/projects/pytenn2016/pytenn2016/rot13.py': pytenn2016-rot13 (<string>)

## ...until we register with the `codecs` module.

In [16]:
from codecs import register
from pytenn2016.encoding import search_function

register(search_function)

In [17]:
from pytenn2016.rot13 import hello

hello()

Hello Rotated World!


# What is this actually useful for?

In [18]:
!cat ../pytenn2016/pyxl.py

# encoding: pyxl

import pyxl.html as html

def hello_html():
    return <html>
             <body>Hello World!</body>
           </html>


In [19]:
import pyxl.codec.register # Activates the pyxl encoding

from pytenn2016.pyxl import hello_html

hello_html()

In [20]:
str(hello_html())

'<html><body>Hello World!</body></html>'

# Custom Encoding Summary

- Encodings registered globally with the `codecs` module.
- Opt-in on a **per-file** basis.
- Only one encoding per file.
- Files must end in `.py`.
- Operates semantically on bytes <-> text layer.

# Import Hooks

<img src="images/importhook.svg" width="1000"/>

## Hy - Lisp Embedded in Python

In [21]:
! cat ../pytenn2016/hy_example.hy

(defn hyfact [n]
  "Lisp in Python!"
  (defn fact-impl [n acc]
    (if (<= n 1)
        acc
      (fact-impl (- n 1) (* acc n))))
  (fact-impl n 1))


In [22]:
import hy  #  Has to come first to ensure that import hook is set.
from pytenn2016.hy_example import hyfact

hyfact(5)

120

# Cython - Pseudo-Python Compiled to C

In [23]:
!cat ../pytenn2016/cython_example.pyx

cpdef cyfact(int n):
    cdef int acc = 1
    cdef int i
    for i in range(1, n + 1):
        acc *= i
    return acc


In [24]:
import pyximport
pyximport.install()

from pytenn2016.cython_example import cyfact

print("cyfact is a %s" % type(cyfact))
cyfact(5)

cyfact is a <class 'builtin_function_or_method'>


120

In [25]:
print("Python Factorial:")
%timeit hyfact(25)

print("\nCython Factorial:")
%timeit cyfact(25)

Python Factorial:
100000 loops, best of 3: 3.38 µs per loop

Cython Factorial:
10000000 loops, best of 3: 43.7 ns per loop


# Import Hook Summary

- Registered Globally on `sys.meta_path`.
- Loader gets to choose how a file is imported.
- No restrictions on file structure.
  - Doesn't even have correspond to a filesystem entry...

# Problem:

Both import hooks and custom encodings rely on some **external** piece of code having been run before a hooked module can be executed.

This makes it hard to ensure that transformations are used correctly/reliably by users.

# What if we just rewrite the code we already have...?

# Bytecode Manipulation

<img src="images/bytecode.svg" width="750"/>

In [26]:
addcode = addtwo.__code__
addcode

<code object addtwo at 0x7fef4c0fa1e0, file "<ipython-input-9-ba723be474f5>", line 1>

In [27]:
from pytenn2016.bytecode import code_attrs

code_attrs(addcode)

{'co_argcount': 1,
 'co_cellvars': (),
 'co_code': b'|\x00\x00d\x01\x00\x17S',
 'co_consts': (None, 2),
 'co_filename': '<ipython-input-9-ba723be474f5>',
 'co_firstlineno': 1,
 'co_flags': 67,
 'co_freevars': (),
 'co_kwonlyargcount': 0,
 'co_lnotab': b'\x00\x01',
 'co_name': 'addtwo',
 'co_names': (),
 'co_nlocals': 1,
 'co_stacksize': 2,
 'co_varnames': ('a',)}

In [28]:
import dis

print("Raw Bytes: %s" % list(addcode.co_code))
print("\nDisassembly:\n")
dis.dis(addcode) 

Raw Bytes: [124, 0, 0, 100, 1, 0, 23, 83]

Disassembly:

  2           0 LOAD_FAST                0 (a)
              3 LOAD_CONST               1 (2)
              6 BINARY_ADD
              7 RETURN_VALUE


### Addition is so 2015...

In [31]:
def replace(l, old, new):
    "Replace all instances of `old` in `l` with `new`"
    out = []
    for elem in l:
        if elem == old:
            out.append(new)
        else:
            out.append(elem)
    return out

addbytes = addcode.co_code
mulbytes = bytes(replace(list(addbytes), 23, 20))

print("Old Disassembly:")
dis.dis(addbytes)
print("\nNew Disassembly:")
dis.dis(mulbytes)

Old Disassembly:
          0 LOAD_FAST                0 (0)
          3 LOAD_CONST               1 (1)
          6 BINARY_ADD
          7 RETURN_VALUE

New Disassembly:
          0 LOAD_FAST                0 (0)
          3 LOAD_CONST               1 (1)
          6 BINARY_MULTIPLY
          7 RETURN_VALUE


# Just overwriting the code won't work...

In [32]:
add.__code__.co_code = mulbytes

AttributeError: readonly attribute

# ...but copying everything else will!

In [33]:
from types import CodeType

mulcode = CodeType(
    addcode.co_argcount,
    addcode.co_kwonlyargcount,
    addcode.co_nlocals,
    addcode.co_stacksize,
    addcode.co_flags,
    mulbytes, # This is our only change.
    addcode.co_consts,
    addcode.co_names,
    addcode.co_varnames,
    addcode.co_filename,
    addcode.co_name,
    addcode.co_firstlineno,
    addcode.co_lnotab,
    addcode.co_freevars,
    addcode.co_cellvars,
)

mulcode

<code object addtwo at 0x7fef2b5cd390, file "<ipython-input-9-ba723be474f5>", line 1>

# We can use the same technique to rebuild a modified function.

In [34]:
from types import FunctionType

multwo = FunctionType(
    mulcode,
    addtwo.__globals__,
    'multwo',
    addtwo.__defaults__,
    addtwo.__closure__,
)

In [35]:
multwo(5)

10

<img src="images/mindblown.gif" width="2000"/>

# Pattern for Bytecode Transformers

- Start with an existing function.
- Extract its `__code__`.
- Apply some transformation.
- Contruct new `code` by copying everything not changed.
- Construct new function from new code object.

In [36]:
from codetransformer import CodeTransformer, pattern
from codetransformer.instructions import *

class ruby_strings(CodeTransformer):
    
    @pattern(LOAD_CONST)
    def _format_bytes(self, instr):
        yield instr
        if not isinstance(instr.arg, bytes):
            return

        # Equivalent to:
        # s.decode('utf-8').format(**locals())
        yield LOAD_ATTR('decode')
        yield LOAD_CONST('utf-8')
        yield CALL_FUNCTION(1)
        yield LOAD_ATTR('format')
        yield LOAD_CONST(locals) 
        yield CALL_FUNCTION(0)
        yield CALL_FUNCTION_KW()

In [37]:
@ruby_strings()
def example(a, b, c):
    return b"a is {a}, b is {b}, c is {c}"

example(1, 2, 3)

'a is 1, b is 2, c is 3'

# More Examples:  Overloaded Exceptions

In [38]:
from codetransformer.transformers.exc_patterns import \
    pattern_matched_exceptions
  
@pattern_matched_exceptions()
def foo():
    try:
        raise ValueError('bar')
    except ValueError('buzz'):
        return 'buzz'
    except ValueError('bar'):
        return 'bar'
    
foo()

'bar'

# More Examples : Overloaded Literals

In [39]:
from codetransformer.transformers.literals import ordereddict_literals

@ordereddict_literals
def make_dictionary(a, b):
    return {a: 1, b: 2}

make_dictionary('a', 'b')

OrderedDict([('a', 1), ('b', 2)])

# More Examples : `numba`

In [40]:
import numba

@numba.jit
def numbafact(x):
    acc = 1
    for i in range(1, x + 1):
        acc *= i
    return acc

numbafact(5)

120

In [41]:
print(list(numbafact.inspect_llvm().values())[0])

; ModuleID = 'numbafact'
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-pc-linux-gnu"

@PyExc_RuntimeError = external global i8
@.const.numbafact = internal constant [10 x i8] c"numbafact\00"
@".const.Fatal_error:_missing__dynfunc.Closure" = internal constant [38 x i8] c"Fatal error: missing _dynfunc.Closure\00"
@.const.missing_Environment = internal constant [20 x i8] c"missing Environment\00"

; Function Attrs: nounwind
define i32 @"__main__.numbafact$1.int64"(i64* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, i8* noalias nocapture readnone %env, i64 %arg.x) #0 {
entry:
  %.82 = icmp sgt i64 %arg.x, 0
  br i1 %.82, label %B29.preheader, label %B45

B29.preheader:                                    ; preds = %entry
  %0 = xor i64 %arg.x, -1
  %1 = icmp sgt i64 %0, -2
  %smax = select i1 %1, i64 %0, i64 -2
  %2 = add i64 %smax, %arg.x
  %backedge.overflow = icmp eq i64 %2, -2
  br i1 %backedge.overflow, label %B29.pr

In [42]:
print("Python Factorial:")
%timeit hyfact(25)

print("\nCython Factorial:")
%timeit cyfact(25)

print("\nNumba Factorial:")
%timeit numbafact(25)

Python Factorial:
100000 loops, best of 3: 3.33 µs per loop

Cython Factorial:
The slowest run took 26.59 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 44.7 ns per loop

Numba Factorial:
The slowest run took 17.20 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 148 ns per loop


# Bytecode Transformer Summary

- Doesn't require external setup.
- Can be applied on a per-function basis like any other decorator.
- Relies on CPython implementation details:
  - Only works in CPython (no PyPy).
  - Significant changes between minor versions.
- Bugs can cause segfaults.

# Thanks!

- **Talk Content:<h3><a href="https://github.com/ssanderson/pytenn2016">https://github.com/ssanderson/pytenn2016</a>
- **Twitter:** [@ssanderson11235](https://twitter.com/ssanderson11235)
- **GitHub:** [ssanderson](github.com/ssanderson)